In [5]:
import tensorflow as tf
data_dir = "/ssd003/projects/pets/datasets"
dataset = tf.keras.preprocessing.text_dataset_from_directory(f"{data_dir}/yelp/business_federated/business_federated/7-Eleven/", labels='inferred')

Found 1787 files belonging to 5 classes.


2021-10-27 12:42:57.162656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-27 12:42:57.163064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-27 12:42:57.163378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-27 12:42:57.163751: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2021-10-27 12:42:57.164086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

In [8]:
data, labels = next(iter(dataset))

In [10]:
data.shape

TensorShape([32])

In [11]:
labels.shape

TensorShape([32])

In [14]:
# As you can see, some of these reviews are... colourful.
data[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'I realize this was mean spirited and I was drunk when I wrote it.......... But I stand by what i said. That dude was a weenie smoking poodle walker.'>